In [ ]:
import pandas as pd
import numpy as np

import re
from unidecode import unidecode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)
from pyDEA.core.data_processing.read_data_from_xls import construct_input_data_instance
from pyDEA.core.models.envelopment_model import EnvelopmentModelInputOriented,EnvelopmentModelOutputOriented
from pyDEA.core.models.envelopment_model_base import EnvelopmentModelBase
from pyDEA.core.models.bound_generators import generate_upper_bound_for_efficiency_score,generate_lower_bound_for_efficiency_score
from pyDEA.core.models.envelopment_model_decorators import DefaultConstraintCreator
import seaborn as sns


from pyDEA.core.models.multiplier_model import MultiplierInputOrientedModel
from pyDEA.core.models.multiplier_model_base import MultiplierModelBase
dados = pd.read_parquet('custo_anual.parquet')

def limpar_nomes(nome):
    nome = unidecode(nome)
    nome = re.sub(r'[^\w\s]', '', nome)  # Remove caracteres especiais
    nome = nome.replace(' ', '_').replace('/', '_')  # Substitui espaços e barras por _
    return nome


In [ ]:

motos = dados[(dados['categoria']=='Moto Comum')&(dados['qtda_abastecimento']>=10)].dropna()
motos['km_por_real'] = motos['distancia_por_ano']/(motos['valor_abastecimento']+motos['valor_manutencao'])

motos['despesas'] =motos['valor_abastecimento'] + motos['valor_manutencao']

dmus = motos['placa'].values
coefs = motos[['valor_manutencao','valor_abastecimento','distancia_por_ano']].values
dc = {dmu:coef for dmu,coef in zip(dmus, coefs)}
dt = construct_input_data_instance(['manutencao','abastecimento','distancia'], dc)
dt.add_output_category('distancia')
dt.add_input_category('manutencao')
dt.add_input_category('abastecimento')

mdl = MultiplierInputOrientedModel()
mdl1 = MultiplierModelBase(dt,1.0e-8,mdl)
sol = mdl1.run()

solucao = []
for dmu in dt.DMU_codes:
    try:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    except:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':0}  
    # aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    solucao.append(aux)

aux = pd.DataFrame(solucao)

xx = pd.merge(motos,aux, how='left', on='placa')
sns.scatterplot(y='distancia_por_ano',x='despesas',hue='eficiencia',data=xx)


In [ ]:
fg600 = dados[(dados['categoria']=='Furgão 600 Kg')&(dados['qtda_abastecimento']>=20)].dropna()
fg600['km_por_real'] = fg600['distancia_por_ano']/(fg600['valor_abastecimento']+fg600['valor_manutencao'])
fg600['despesas'] =fg600['valor_abastecimento'] + fg600['valor_manutencao']

dmus = fg600['placa'].values
coefs = fg600[['valor_manutencao','valor_abastecimento','distancia_por_ano']].values
dc = {dmu:coef for dmu,coef in zip(dmus, coefs)}
dt = construct_input_data_instance(['manutencao','abastecimento','distancia'], dc)
dt.add_output_category('distancia')
dt.add_input_category('manutencao')
dt.add_input_category('abastecimento')

mdl = MultiplierInputOrientedModel()
mdl1 = MultiplierModelBase(dt,1.0e-8,mdl)
sol = mdl1.run()

solucao = []
for dmu in dt.DMU_codes:
    try:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    except:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':0}  
    # aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    solucao.append(aux)

aux = pd.DataFrame(solucao)

xx = pd.merge(fg600,aux, how='left', on='placa')

sns.scatterplot(y='distancia_por_ano',x='despesas',hue='eficiencia',data=xx)

In [70]:
grupos = dados.groupby(['cto_responsavel','categoria']).agg(distancia=('distancia_por_ano','sum'),combustivel=('valor_abastecimento','sum'),manutencao=('valor_manutencao','sum'))
tabela = grupos.reset_index()
tabela_pivotada = tabela.pivot(index='cto_responsavel', columns='categoria', values=['distancia', 'combustivel', 'manutencao'])

# Ajustar os nomes das colunas
tabela_pivotada.columns = [f"{metric}_{categoria}" for metric, categoria in tabela_pivotada.columns]

# Resetar o índice para transformar `cto_responsavel` em uma coluna normal
tabela_pivotada = tabela_pivotada.reset_index()

# Aplicar a função aos nomes das colunas
tabela_pivotada.columns = [limpar_nomes(col) for col in tabela_pivotada.columns]
tabela = tabela_pivotada.replace({np.nan:0})
tabela = tabela[['cto_responsavel', 'distancia_Caminhao_12T', 'distancia_Caminhao_3T',
       'distancia_Caminhao_75T', 'distancia_Carreta_19T',
       'distancia_Carreta_23T', 'distancia_Furgao_1000_kg',
       'distancia_Furgao_1500_kg', 'distancia_Furgao_400_kg',
       'distancia_Furgao_600_Kg', 'distancia_Moto_Comum',
       'distancia_Moto_Trail', 
       'combustivel_Caminhao_12T', 'combustivel_Caminhao_3T',
       'combustivel_Caminhao_75T', 'combustivel_Carreta_19T',
       'combustivel_Carreta_23T', 'combustivel_Furgao_1000_kg',
       'combustivel_Furgao_1500_kg', 'combustivel_Furgao_400_kg',
       'combustivel_Furgao_600_Kg', 'combustivel_Moto_Comum',
       'combustivel_Moto_Trail', 
       'manutencao_Caminhao_12T', 'manutencao_Caminhao_3T',
       'manutencao_Caminhao_75T', 'manutencao_Carreta_19T',
       'manutencao_Carreta_23T', 'manutencao_Furgao_1000_kg',
       'manutencao_Furgao_1500_kg', 'manutencao_Furgao_400_kg',
       'manutencao_Furgao_600_Kg', 'manutencao_Moto_Comum',
       'manutencao_Moto_Trail']]

manut = ['manutencao_Caminhao_12T', 'manutencao_Caminhao_3T',
       'manutencao_Caminhao_75T', 'manutencao_Carreta_19T',
       'manutencao_Carreta_23T', 'manutencao_Furgao_1000_kg',
       'manutencao_Furgao_1500_kg', 'manutencao_Furgao_400_kg',
       'manutencao_Furgao_600_Kg', 'manutencao_Moto_Comum',
       'manutencao_Moto_Trail']

combs = ['combustivel_Caminhao_12T', 'combustivel_Caminhao_3T',
       'combustivel_Caminhao_75T', 'combustivel_Carreta_19T',
       'combustivel_Carreta_23T', 'combustivel_Furgao_1000_kg',
       'combustivel_Furgao_1500_kg', 'combustivel_Furgao_400_kg',
       'combustivel_Furgao_600_Kg', 'combustivel_Moto_Comum',
       'combustivel_Moto_Trail']

for m,c in zip(manut,combs):
    cat = "_".join(m.split("_")[1:])
    tabela[f"despesas_{cat}"] = tabela[m]+tabela[c]


In [83]:
colunas =['distancia_Furgao_1000_kg',
       'distancia_Furgao_1500_kg', 'distancia_Furgao_400_kg',
       'distancia_Furgao_600_Kg', 'distancia_Moto_Comum',
       'distancia_Moto_Trail',
       
       'despesas_Furgao_1000_kg',
       'despesas_Furgao_1500_kg', 'despesas_Furgao_400_kg',
       'despesas_Furgao_600_Kg', 'despesas_Moto_Comum', 'despesas_Moto_Trail']

coefs = tabela[colunas].values


dmus = tabela['cto_responsavel'].values
dc = {dmu:coef for dmu,coef in zip(dmus, coefs)}
dt = construct_input_data_instance(colunas, dc)

for cl in ['despesas_Furgao_1000_kg',
       'despesas_Furgao_1500_kg', 'despesas_Furgao_400_kg',
       'despesas_Furgao_600_Kg', 'despesas_Moto_Comum', 'despesas_Moto_Trail']:
       dt.add_input_category(cl)

for cl in ['distancia_Furgao_1000_kg',
       'distancia_Furgao_1500_kg', 'distancia_Furgao_400_kg',
       'distancia_Furgao_600_Kg', 'distancia_Moto_Comum',
       'distancia_Moto_Trail']:
       dt.add_output_category(cl)


mdl = MultiplierInputOrientedModel()
mdl1 = MultiplierModelBase(dt,1.0e-8,mdl)
sol = mdl1.run()

/Users/1moi6/GitHub/Python/tccs/tccs/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/1moi6/GitHub/Python/tccs/tccs/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/z6/n6t25pkx2nl6t75_c5d0rm0h0000gn/T/5dea416f32574ef5a615647cd06fc4d5-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/z6/n6t25pkx2nl6t75_c5d0rm0h0000gn/T/5dea416f32574ef5a615647cd06fc4d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 52 COLUMNS
At line 536 RHS
At line 584 BOUNDS
At line 597 ENDATA
Problem MODEL has 47 rows, 12 columns and 477 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 42 (-5) rows, 12 (0) columns and 472 (-5) elements
0  Obj 0.01820324 Primal inf 6.4826554e-08 (1) Dual inf 37777116 (6)
11  Obj 0.95591715
Optimal - objective value 0.95591715
After Postsolve, objective 0.95591715, infeasibilities - dual 0 (0), primal 0.11365888 (4)
Presolved model w

In [84]:
solucao = []
for dmu in dt.DMU_codes:
    try:
        aux = {'cto_responsavel':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    except:
        aux = {'cto_responsavel':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':0}  
    # aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    solucao.append(aux)

aux = pd.DataFrame(solucao)

xx = pd.merge(tabela,aux, how='left', on='cto_responsavel')

# sns.scatterplot(y='distancia_por_ano',x='despesas',hue='eficiencia',data=xx)

In [85]:
xx[['cto_responsavel','eficiencia']].sort_values(by='eficiencia',ascending=False)

,cto_responsavel,eficiencia
0,CETO/ACR,1.000000
1,CETO/AP,1.000000
2,CETO/RR,1.000000
3,CTO ARACAJU,1.000000
4,CTO BAURU - SPI,1.000000
9,CTO CAMPINAS - SPI,1.000000
13,CTO CURITIBA,1.000000
12,CTO CUIABA,1.000000
11,CTO CENTRO - SPM,1.000000
10,CTO CAMPO GRANDE,1.000000


In [92]:
dados[dados['cto_responsavel']=='CTO BRASILIA']['categoria'].value_counts()

categoria
Furgão 600 Kg     285
Moto Comum        273
Moto Trail         73
Furgão 1500 kg     59
Furgão 400 kg      21
Furgão 1000 kg     10
Caminhão 3T         3
Caminhão 7,5T       3
Caminhão 12T        2
Name: count, dtype: int64

In [ ]:
solucao = []
for dmu in dt.DMU_codes:
    try:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    except:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':0}  
    # aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    solucao.append(aux)

aux = pd.DataFrame(solucao)

xx = pd.merge(fg600,aux, how='left', on='placa')

sns.scatterplot(y='distancia_por_ano',x='despesas',hue='eficiencia',data=xx)

In [ ]:
sns.scatterplot(x = 'km_por_real',y='eficiencia',data=xx)

In [ ]:
xx.sort_values(by='eficiencia',ascending=False)

In [ ]:
sol._print_for_one_dmu('dmu_814')

In [ ]:
sol.get_efficiency_score('dmu_814')

In [ ]:
mdl = EnvelopmentModelOutputOriented(lower_bound_generator=generate_lower_bound_for_efficiency_score)
mdl1 = EnvelopmentModelBase(dt,mdl,DefaultConstraintCreator())
sol = mdl1.run()

In [ ]:
mdl = EnvelopmentModelInputOriented(upper_bound_generator=generate_upper_bound_for_efficiency_score)
mdl1 = EnvelopmentModelBase(dt,mdl,DefaultConstraintCreator())
sol = mdl1.run()



In [ ]:
solucao = []
for dmu in dt.DMU_codes:
    try:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    except:
        aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':0}  
    # aux = {'placa':dt.get_dmu_user_name(dmu),'dmu':dmu,'eficiencia':sol.get_efficiency_score(dmu)}
    solucao.append(aux)

aux = pd.DataFrame(solucao)

xx = pd.merge(motos,aux, how='left', on='placa')


In [ ]:
26789.0/(3971.33/5)

In [ ]:
xx.sort_values(by='distancia',ascending=True).head(100)

In [ ]:
xx

In [ ]:
mdl.get_input_variable_coefficient(obj_variable='E1',input_category='')

In [ ]:
sol.add_lambda_variables()

In [ ]:
xx

In [ ]:
pd.merge(dados,xx, how='left', on='placa')